# `SENTIMENTAL ANALYSIS`

#### `IMPORTIN LIBRARIES`

In [14]:
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### `IMPORTING DATASET`

In [15]:
dataset=pd.read_csv("train.csv")
dataset.drop(["id"],axis=1)

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
31957,0,ate @user isz that youuu?ðððððð...
31958,0,to see nina turner on the airwaves trying to...
31959,0,listening to sad songs on a monday morning otw...
31960,1,"@user #sikh #temple vandalised in in #calgary,..."


In [16]:
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


### `PRE-PROCESSING`

##### `PRE PROCESSING VARIABLE SETTING`

In [17]:
# def clean_text(text):
#     '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
#     and remove words containing numbers.'''
#     text = str(text).lower()
#     text = re.sub('\[.*?\]', ' ', text)
#     text = re.sub('https?://\S+|www\.\S+', ' ', text)
#     text = re.sub('<.*?>+', ' ', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
#     text = re.sub('\n', ' ', text)
#     text = re.sub('\w*\d\w*', ' ', text)
#     return text

In [18]:
dataset['tweet'] = dataset['tweet'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

In [19]:
dataset["tweet"]=dataset["tweet"].apply(lambda x: " ".join([w for w in x.split() if w not in stopwords.words("english")]))
dataset

,id,label,tweet
0,1,0,user father dysfunctional selfish drags kids d...
1,2,0,user user thanks lyft credit cant use cause do...
2,3,0,bihday majesty
3,4,0,model love u take u time ur
4,5,0,factsguide society motivation
...,...,...,...
31957,31958,0,ate user isz youuu
31958,31959,0,see nina turner airwaves trying wrap mantle ge...
31959,31960,0,listening sad songs monday morning otw work sad
31960,31961,1,user sikh temple vandalised calgary wso condem...


In [20]:
# Stemming
st=PorterStemmer()
dataset["tweet"]=dataset['tweet'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [21]:
# Lemmatizing

dataset['tweet']=dataset['tweet'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
dataset

,id,label,tweet
0,1,0,user father dysfunct selfish drag kid dysfunct...
1,2,0,user user thank lyft credit cant use caus dont...
2,3,0,bihday majesti
3,4,0,model love u take u time ur
4,5,0,factsguid societi motiv
...,...,...,...
31957,31958,0,ate user isz youuu
31958,31959,0,see nina turner airwav tri wrap mantl genuin h...
31959,31960,0,listen sad song monday morn otw work sad
31960,31961,1,user sikh templ vandalis calgari wso condemn act


### `SPLITTING DATA FOR TRAINING AND TESTING`

#### `LSTM MODEL`

In [22]:
max_words = 1000
max_len=50

def tokenize_pad_sequences(text):
    tokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')
    tokenizer.fit_on_texts(text)
    X = tokenizer.texts_to_sequences(text)
    X = pad_sequences(X, padding='post', maxlen=max_len)
    return X, tokenizer

print('Before Tokenization & Padding \n', dataset['tweet'][0])
X, tokenizer = tokenize_pad_sequences(dataset['tweet'])
print('After Tokenization & Padding \n', X[0])

Before Tokenization & Padding 
 user father dysfunct selfish drag kid dysfunct run
After Tokenization & Padding 
 [   1   18 2514 1821  129  193    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [23]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_words, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 128)           640000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 895,194
Trainable params: 895,194
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
from sklearn.model_selection import train_test_split
Y=pd.get_dummies(dataset["label"]).values
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(25569, 50) (25569, 2)
(6393, 50) (6393, 2)


In [26]:
batch_size = 32
model.fit(X_train, y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
800/800 - 55s - loss: 0.2332 - accuracy: 0.9301 - 55s/epoch - 69ms/step
Epoch 2/7
800/800 - 55s - loss: 0.2336 - accuracy: 0.9301 - 55s/epoch - 68ms/step
Epoch 3/7
800/800 - 56s - loss: 0.2512 - accuracy: 0.9301 - 56s/epoch - 70ms/step
Epoch 4/7
800/800 - 54s - loss: 0.2541 - accuracy: 0.9301 - 54s/epoch - 68ms/step
Epoch 5/7
800/800 - 54s - loss: 0.2519 - accuracy: 0.9301 - 54s/epoch - 68ms/step
Epoch 6/7
800/800 - 54s - loss: 0.2485 - accuracy: 0.9301 - 54s/epoch - 67ms/step
Epoch 7/7
800/800 - 53s - loss: 0.2437 - accuracy: 0.9301 - 53s/epoch - 67ms/step


In [27]:
twt = [["good day"]]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=50, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 - 4s - 4s/epoch - 4s/step
negative
